In [ ]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from sklearn.metrics.pairwise import cosine_similarity

from Utils.arch import *
from Utils.helper import *
from Utils.loss import *
from Utils.args import img_size

In [ ]:
iml=sorted([os.path.abspath(x) for x in glob.glob('data/**/*.jpg',recursive=True)])
batch_size = 50
thresh = 0.75
min_samp = 3

device = torch.device('cuda:0')
file = 'checkpoints/ddp_model_12420.pth'

In [ ]:
def tag(i,clas):
    if visited[i] == 1:
        return
    
    selected = (mat[i]>thresh).nonzero()[0]
    visited[i] = 1
    clss[i] = clas
    
    for entry in selected:
        tag(entry,clas)

In [ ]:
if 'ddp' in file:
    net=torch.nn.DataParallel(Network(device))
else:
    net = Network(device)


x = torch.load(file)
net.load_state_dict(x)

if 'ddp' in file:
    net = net.module

for p in net.parameters():
    p.requires_grad = False

net.to(device)


In [ ]:
output_dict={}
for batch_paths in [ iml[i:i+batch_size] for i in range(0,len(iml),batch_size)]:

    batch=[]
    print(len(batch_paths))
    for path in batch_paths:
        img=cv2.imread(path)
        if img.shape[:2]!=img_size:
            img=cv2.resize(img,(img_size[1],img_size[0]))
        batch.append(img)

    batch = np.array(batch)
    batch = torch.from_numpy(batch)
    batch = batch.permute(0, 3, 1, 2)
    batch = batch.to(device).float()


    batch_embed = net(batch)

    batch_embed = batch_embed[0].cpu().detach().numpy()

    for path,embed in zip(batch_paths,batch_embed):
        output_dict[path]=embed


In [ ]:
embeds = np.array(list(output_dict.values()))
names = np.array(list(output_dict.keys()))

mat = cosine_similarity(embeds)

visited = np.zeros((len(mat)))
clss = np.zeros((len(mat)))

clas=0
for i in range(len(mat)):
    
    if visited[i] == 0:
        clas+=1
        tag(i,clas)
        
to_set = []
for entry in np.unique(clss):
    if sum(clss == entry) <min_samp:
        to_set.append(entry)

for entry in to_set:
    clss[clss==entry] = 0

print(len(np.unique(clss)))

In [ ]:
for img,cls in zip(names,clss):
    try:
        os.makedirs('cluster/'+str(int(cls)))
    except:
        pass

    try:
        os.symlink(img,'cluster/'+str(int(cls))+'/'+img.split(os.sep)[-1])
        # shutil.copy(img,'cluster/'+str(int(cls))+'/')
    except FileExistsError:
        pass